In [1]:
import pandas as pd
import os
os.chdir('../') #Esto sube un nivel desde 'Notebooks' a 'Proyect_amazon'(donde esta Functions.py)
import sys
# permite que Python encuentre e importe el módulo 'Functions' desde el directorio actual
sys.path.append(os.getcwd())
import Functions as fs

In [2]:
df_ventas = pd.read_csv('Amazon Sale Report.csv', low_memory=False)
df_ventas.head(2)

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN


In [3]:
fs.data_characteristics(df_ventas)

,column,type data,total nulls,percentage
0,index,int64,0,0.0%
1,Order ID,object,0,0.0%
2,Date,object,0,0.0%
3,Status,object,0,0.0%
4,Fulfilment,object,0,0.0%
5,Sales Channel,object,0,0.0%
6,ship-service-level,object,0,0.0%
7,Style,object,0,0.0%
8,SKU,object,0,0.0%
9,Category,object,0,0.0%


**Eliminacion de columnas**

Se comenzara eliminando las columnas irrelevantes del conjunto de datos según lo mencionado en la conclusión del análisis preliminar.

In [4]:
columnas_innecesarias = ['index','promotion-ids','fulfilled-by','Unnamed: 22']
df_ventas.drop(columnas_innecesarias,axis=1,inplace=True)

In [5]:
df_ventas.dtypes

Order ID               object
Date                   object
Status                 object
Fulfilment             object
Sales Channel          object
ship-service-level     object
Style                  object
SKU                    object
Category               object
Size                   object
ASIN                   object
Courier Status         object
Qty                     int64
currency               object
Amount                float64
ship-city              object
ship-state             object
ship-postal-code      float64
ship-country           object
B2B                      bool
dtype: object

In [6]:
#Cambio tipos de datos de la columna Date y ship-postal-code
df_ventas['Date'] = pd.to_datetime(df_ventas['Date'],format="%m-%d-%y", errors='coerce')
df_ventas['ship-postal-code'] = df_ventas['ship-postal-code'].astype('object')

#Se transformaron los valores a cadena (.astype(str)) para poder remplazar, porque sino da error
df_ventas['ship-postal-code'] = df_ventas['ship-postal-code'].astype(str).str.replace('.0', '')

In [7]:
df_ventas.sample()

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,B2B
18424,171-3463574-5638707,2022-04-19,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,J0341,J0341-DR-M,Western Dress,M,B099NQQ79L,Shipped,1,INR,744.0,VIRUDHACHALAM,TAMIL NADU,606001,IN,False


**Eliminacion de duplicados**

In [8]:
df_ventas.duplicated().sum()

6

In [9]:
# Identificar filas duplicadas considerando todas las columnas
duplicated_rows = df_ventas[df_ventas.duplicated(keep=False)]
duplicated_rows

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,B2B
30660,406-0372545-6086735,2022-04-12,Cancelled,Amazon,Amazon.in,Expedited,SET197,SET197-KR-NP-L,Set,L,B08B3YPD63,Cancelled,0,NaN,NaN,Siliguri,WEST BENGAL,734008,IN,False
30661,406-0372545-6086735,2022-04-12,Cancelled,Amazon,Amazon.in,Expedited,SET197,SET197-KR-NP-L,Set,L,B08B3YPD63,Cancelled,0,NaN,NaN,Siliguri,WEST BENGAL,734008,IN,False
41291,408-0373839-4433120,2022-04-05,Cancelled,Amazon,Amazon.in,Expedited,JNE3501,JNE3501-KR-M,kurta,M,B08MYVCDB4,Cancelled,0,NaN,NaN,BENGALURU,KARNATAKA,560072,IN,False
41292,408-0373839-4433120,2022-04-05,Cancelled,Amazon,Amazon.in,Expedited,JNE3501,JNE3501-KR-M,kurta,M,B08MYVCDB4,Cancelled,0,NaN,NaN,BENGALURU,KARNATAKA,560072,IN,False
79844,171-9628368-5329958,2022-05-07,Cancelled,Amazon,Amazon.in,Expedited,J0329,J0329-KR-L,kurta,L,B09KXRB7JV,Cancelled,0,NaN,NaN,ERNAKULAM,KERALA,682017,IN,False
79845,171-9628368-5329958,2022-05-07,Cancelled,Amazon,Amazon.in,Expedited,J0329,J0329-KR-L,kurta,L,B09KXRB7JV,Cancelled,0,NaN,NaN,ERNAKULAM,KERALA,682017,IN,False
85790,171-3249942-2207542,2022-05-03,Shipped,Amazon,Amazon.in,Expedited,SET323,SET323-KR-NP-XL,Set,XL,B09NDKKM7P,Shipped,1,INR,939.0,PUNE,MAHARASHTRA,411057,IN,False
85791,171-3249942-2207542,2022-05-03,Shipped,Amazon,Amazon.in,Expedited,SET323,SET323-KR-NP-XL,Set,XL,B09NDKKM7P,Shipped,1,INR,939.0,PUNE,MAHARASHTRA,411057,IN,False
86418,405-8669298-3850736,2022-05-03,Shipped,Amazon,Amazon.in,Expedited,MEN5025,MEN5025-KR-XXXL,kurta,3XL,B08YYQS8FH,Shipped,1,INR,533.0,GHAZIABAD,UTTAR PRADESH,201010,IN,False
86419,405-8669298-3850736,2022-05-03,Shipped,Amazon,Amazon.in,Expedited,MEN5025,MEN5025-KR-XXXL,kurta,3XL,B08YYQS8FH,Shipped,1,INR,533.0,GHAZIABAD,UTTAR PRADESH,201010,IN,False


In [10]:
#Se eliminan las filas duplicadas
df_ventas.drop_duplicates(inplace=True)

In [11]:
frecuencia = df_ventas['Order ID'].value_counts()
frecuencia[frecuencia > 1].head()

Order ID
403-4984515-8861958    12
171-5057375-2831560    12
404-9932919-6662730    11
403-0173977-3041148    11
408-3317403-1729937    10
Name: count, dtype: int64

Al visualizar los valores repetidos de *Order ID* llegamos a la conclusion de que se trata de varias ventas que pertenecen a la misma orden.

In [12]:
df_ventas[df_ventas['Order ID']== "403-4984515-8861958"].head(3)

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,B2B
31737,403-4984515-8861958,2022-04-11,Shipped,Amazon,Amazon.in,Expedited,JNE3792,JNE3792-KR-XXXL,kurta,3XL,B09KXNWRSB,Shipped,1,INR,432.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31738,403-4984515-8861958,2022-04-11,Shipped,Amazon,Amazon.in,Expedited,JNE3764,JNE3764-KR-XXXL,kurta,3XL,B09K3SSDSZ,Shipped,1,INR,487.0,SOLAPUR,MAHARASHTRA,413002,IN,False
31739,403-4984515-8861958,2022-04-11,Shipped,Amazon,Amazon.in,Expedited,JNE3503,JNE3503-KR-XXXL,kurta,3XL,B08RP2BV5Q,Shipped,1,INR,318.0,SOLAPUR,MAHARASHTRA,413002,IN,False


In [13]:
#Filtramos por Order ID y una fecha diferente, ya que una orden puede pertencer a 1 dia especifico
df_ventas[(df_ventas['Order ID']== "403-4984515-8861958") & (df_ventas['Date']!="2022-04-11")].head(2)

,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,B2B
